![alt text](../utils/conve.png)

In [1]:
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import sklearn
import pprint
import itertools

import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

from data import WN18

In [2]:
class Config:
    n_epochs = 100
    batch_size = 128
    embed_dim = 200

In [3]:
"""
e: entity
s: subject
p: predicate
o: object
"""

def read_triples(path):
    triples = []
    with open(path, 'rt') as f:
        for line in f.readlines():
            s, p, o = line.split()
            triples += [(s.strip(), p.strip(), o.strip())]
    return triples


def load_triple():
    WN18.download()
    triples_tr = read_triples('../data/WN18/wn18/train.txt')
    triples_va = read_triples('../data/WN18/wn18/valid.txt')
    triples_te = read_triples('../data/WN18/wn18/test.txt')
    
    triples_all = triples_tr + triples_va + triples_te
    
    return triples_all, triples_tr, triples_va, triples_te


def build_vocab(triples):
    params = {}
    
    e_set = {s for (s, p, o) in triples} | {o for (s, p, o) in triples}
    p_set = {p for (s, p, o) in triples}

    params['e_vocab_size'] = len(e_set)
    params['p_vocab_size'] = len(p_set)

    e2idx = {e: idx for idx, e in enumerate(sorted(e_set))}
    p2idx = {p: idx for idx, p in enumerate(sorted(p_set))}
    
    return e2idx, p2idx, params


def build_multi_label(triples_tr):
    s2p2o = {}
    for (_s, _p, _o) in triples_tr:
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
        if s not in s2p2o:
            s2p2o[s] = {p: [o]}
        if p not in s2p2o[s]:
            s2p2o[s][p] = [o]
        if o not in s2p2o[s][p]:
            s2p2o[s][p].append(o)
    return s2p2o


def next_train_batch(triples_tr, e2idx, p2idx, s2p2o):
    for i in range(0, len(triples_tr), Config.batch_size):
        _triples_tr = triples_tr[i : i+Config.batch_size]
        x_s = np.asarray([e2idx[s] for (s, p, o) in _triples_tr], dtype=np.int32)
        x_p = np.asarray([p2idx[p] for (s, p, o) in _triples_tr], dtype=np.int32)
        y = []
        for (_s, _p, _o) in _triples_tr:
            s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
            temp = np.zeros([len(e2idx)])
            temp[s2p2o[s][p]] = 1.
            y.append(temp)
        y = np.asarray(y)
        yield x_s, x_p, y


def train_input_fn(triples_tr, e2idx, p2idx, s2p2o):
    dataset = tf.data.Dataset.from_generator(
        lambda: next_train_batch(sklearn.utils.shuffle(triples_tr),
                                 e2idx,
                                 p2idx,
                                 s2p2o),
        (tf.int32, tf.int32, tf.float32),
        (tf.TensorShape([None]),
         tf.TensorShape([None]),
         tf.TensorShape([None, len(e2idx)])))
    iterator = dataset.make_one_shot_iterator()
    x_s, x_p, y = iterator.get_next()
    return {'s': x_s, 'p': x_p}, y

In [4]:
def o_next_batch(eval_triples,
                 entity_to_idx, 
                 predicate_to_idx):
    for _i, (s, p, o) in enumerate(eval_triples):
        s_idx, p_idx = entity_to_idx[s], predicate_to_idx[p]
        yield np.atleast_1d(s_idx), np.atleast_1d(p_idx)


def o_input_fn(eval_triples,
               entity_to_idx, 
               predicate_to_idx):
    dataset = tf.data.Dataset.from_generator(
        lambda: o_next_batch(eval_triples,
                             entity_to_idx, 
                             predicate_to_idx),
        (tf.int32, tf.int32),
        (tf.TensorShape([None,]),
         tf.TensorShape([None,])))
    iterator = dataset.make_one_shot_iterator()
    s, p = iterator.get_next()
    return {'s': s, 'p': p}


def evaluate_rank(model,
                  valid_triples,
                  test_triples,
                  all_triples,
                  entity_to_idx,
                  predicate_to_idx):

    for eval_name, eval_triples in [('test', test_triples)]:
        _scores_o = list(model.predict(
            lambda: o_input_fn(eval_triples,
                               entity_to_idx, 
                               predicate_to_idx)))

        ScoresO = np.reshape(_scores_o, [len(eval_triples), len(entity_to_idx)])
        ranks_o, filtered_ranks_o = [], []
    
        for _i, ((s, p, o), scores_o) in enumerate(zip(eval_triples, ScoresO)):
            s_idx, p_idx, o_idx = entity_to_idx[s], predicate_to_idx[p], entity_to_idx[o]
            
            ranks_o += [1 + np.argsort(np.argsort(- scores_o))[o_idx]]
            
            filtered_scores_o = scores_o.copy()
            rm_idx_o = [entity_to_idx[fo] for (fs, fp, fo) in all_triples if fs == s and fp == p and fo != o]
            filtered_scores_o[rm_idx_o] = - np.inf
            filtered_ranks_o += [1 + np.argsort(np.argsort(- filtered_scores_o))[o_idx]]

        for setting_name, setting_ranks in [('Raw', ranks_o), ('Filtered', filtered_ranks_o)]:
            mean_rank = np.mean(setting_ranks)
            print('[{}] {} Mean Rank: {}'.format(eval_name, setting_name, mean_rank))
            for k in [1, 3, 5, 10]:
                hits_at_k = np.mean(np.asarray(setting_ranks) <= k) * 100
                print('[{}] {} Hits@{}: {}'.format(eval_name, setting_name, k, hits_at_k))

In [ ]:
def forward(features, mode, params):
    batch_sz = tf.shape(features['s'])[0]
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    
    e_embed = tf.get_variable('e_embed',
                              [params['e_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    p_embed = tf.get_variable('p_embed',
                              [params['p_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    
    s = tf.nn.embedding_lookup(e_embed, features['s'])
    p = tf.nn.embedding_lookup(p_embed, features['p'])
    
    x = tf.concat([tf.reshape(s, [batch_sz, 10, 20, 1]),
                   tf.reshape(p, [batch_sz, 10, 20, 1])], 1)
    
    x = tf.layers.dropout(x, 0.2, training=is_training)
    x = tf.layers.conv2d(x, 32, (3,3), activation=tf.nn.elu)
    
    x = tf.layers.flatten(x)
    x = tf.layers.dropout(x, 0.5, training=is_training)
    x = tf.layers.dense(x, Config.embed_dim, tf.nn.elu)
    
    logits = tf.matmul(x, e_embed, transpose_b=True)
    return logits
    
    
def model_fn(features, labels, mode, params):
    logits = forward(features, mode, params)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        tf.logging.info('\n'+pprint.pformat(tf.trainable_variables()))
        
        global_step = tf.train.get_global_step()
        
        loss_op = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,
                                                                        labels=labels))
        
        lr_op = tf.train.exponential_decay(5e-3, global_step, 1100, 0.97)
        
        train_op = tf.train.AdamOptimizer(lr_op).minimize(loss_op,
                                                          global_step = global_step)
        
        lth = tf.train.LoggingTensorHook({'lr': lr_op,}, every_n_iter=100)
        
        return tf.estimator.EstimatorSpec(mode = mode,
                                          loss = loss_op,
                                          train_op = train_op,
                                          training_hooks = [lth])
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions = tf.sigmoid(logits))

In [ ]:
triples_all, triples_tr, triples_va, triples_te = load_triple()
e2idx, p2idx, params = build_vocab(triples_all)
s2p2o = build_multi_label(triples_tr)

model = tf.estimator.Estimator(model_fn,
                               model_dir = './ConvE',
                               config = tf.estimator.RunConfig(keep_checkpoint_max=1),
                               params = params)

for n_epoch in range(Config.n_epochs):
    model.train(lambda: train_input_fn(triples_tr, e2idx, p2idx, s2p2o))
    evaluate_rank(model,
                  triples_va,
                  triples_te,
                  triples_all,
                  e2idx,
                  p2idx,)

Files Already Downloaded
INFO:tensorflow:Using config: {'_model_dir': './ConvE', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x114dc5128>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(10368, 200) dty

INFO:tensorflow:lr = 0.0046511344 (43.806 sec)
INFO:tensorflow:global_step/sec: 2.30717
INFO:tensorflow:loss = 2088.167, step = 2713 (43.343 sec)
INFO:tensorflow:lr = 0.0046382733 (43.343 sec)
INFO:tensorflow:global_step/sec: 2.28805
INFO:tensorflow:loss = 1835.1488, step = 2813 (43.705 sec)
INFO:tensorflow:lr = 0.0046254476 (43.706 sec)
INFO:tensorflow:global_step/sec: 2.28879
INFO:tensorflow:loss = 1633.5331, step = 2913 (43.691 sec)
INFO:tensorflow:lr = 0.0046126572 (43.691 sec)
INFO:tensorflow:global_step/sec: 2.2566
INFO:tensorflow:loss = 1628.2766, step = 3013 (44.314 sec)
INFO:tensorflow:lr = 0.0045999023 (44.314 sec)
INFO:tensorflow:global_step/sec: 2.23741
INFO:tensorflow:loss = 1604.3787, step = 3113 (44.695 sec)
INFO:tensorflow:lr = 0.004587183 (44.695 sec)
INFO:tensorflow:global_step/sec: 2.19833
INFO:tensorflow:loss = 1498.2687, step = 3213 (45.489 sec)
INFO:tensorflow:lr = 0.0045744986 (45.489 sec)
INFO:tensorflow:global_step/sec: 2.2724
INFO:tensorflow:loss = 1682.559, s

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ConvE/model.ckpt-5530
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 5531 into ./ConvE/model.ckpt.
INFO:tensorflow:loss = 523.9213, step = 5531
INFO:tensorflow:lr = 0.0042901053
INFO:tensorflow:global_step/sec: 2.44517
INFO:tensorflow:loss = 562.87213, step = 5631 (40.898 sec)
INFO:tensorflow:lr = 0.0042782426 (40.898 sec)
INFO:tensorflow:global_step/sec: 2.3509
INFO:tensorflow:loss = 625.339, step = 5731 (42.537 sec)
INFO:tensorflow:lr = 0.0042664125 (42.537 sec)
INFO:tensorflow:global_step/sec: 2.47675
INFO:tensorflow:loss = 514.62317, step = 5831 (40.376 sec)
INFO:tensorflow:lr = 0.004254615 (40.376 sec)
INFO:tensorflow:global_step/sec: 2.25154
INFO:tensorflow:loss = 585.23315, step = 5931 (44.414 sec)
INFO:tensorflow:lr = 0.00424285 (44.414 sec)
I

INFO:tensorflow:loss = 434.61267, step = 8843 (44.296 sec)
INFO:tensorflow:lr = 0.0039141625 (44.296 sec)
INFO:tensorflow:Saving checkpoints for 8848 into ./ConvE/model.ckpt.
INFO:tensorflow:Loss for final step: 6.074156.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ConvE/model.ckpt-8848
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[test] Raw Mean Rank: 1108.9576
[test] Raw Hits@1: 20.080000000000002
[test] Raw Hits@3: 31.319999999999997
[test] Raw Hits@5: 36.78
[test] Raw Hits@10: 43.28
[test] Filtered Mean Rank: 1092.9362
[test] Filtered Hits@1: 24.44
[test] Filtered Hits@3: 35.06
[test] Filtered Hits@5: 39.739999999999995
[test] Filtered Hits@10: 45.34
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variab

INFO:tensorflow:lr = 0.0036606716 (39.972 sec)
INFO:tensorflow:global_step/sec: 2.36582
INFO:tensorflow:loss = 268.00146, step = 11361 (42.269 sec)
INFO:tensorflow:lr = 0.0036505493 (42.269 sec)
INFO:tensorflow:global_step/sec: 2.29941
INFO:tensorflow:loss = 243.21667, step = 11461 (43.489 sec)
INFO:tensorflow:lr = 0.003640455 (43.489 sec)
INFO:tensorflow:global_step/sec: 2.3198
INFO:tensorflow:loss = 202.23915, step = 11561 (43.107 sec)
INFO:tensorflow:lr = 0.0036303883 (43.108 sec)
INFO:tensorflow:global_step/sec: 2.43784
INFO:tensorflow:loss = 227.43546, step = 11661 (41.020 sec)
INFO:tensorflow:lr = 0.0036203496 (41.020 sec)
INFO:tensorflow:global_step/sec: 2.41142
INFO:tensorflow:loss = 242.43773, step = 11761 (41.470 sec)
INFO:tensorflow:lr = 0.0036103388 (41.470 sec)
INFO:tensorflow:global_step/sec: 2.29889
INFO:tensorflow:loss = 223.79132, step = 11861 (43.499 sec)
INFO:tensorflow:lr = 0.0036003552 (43.499 sec)
INFO:tensorflow:global_step/sec: 2.27257
INFO:tensorflow:loss = 227

...

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ConvE/model.ckpt-97328
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 97329 into ./ConvE/model.ckpt.
INFO:tensorflow:loss = 0.5394928, step = 97329
INFO:tensorflow:lr = 0.0003377024
INFO:tensorflow:global_step/sec: 2.63128
INFO:tensorflow:loss = 8.653192, step = 97429 (38.006 sec)
INFO:tensorflow:lr = 0.00033676863 (38.005 sec)
INFO:tensorflow:global_step/sec: 2.64548
INFO:tensorflow:loss = 0.38456598, step = 97529 (37.800 sec)
INFO:tensorflow:lr = 0.00033583737 (37.801 sec)
INFO:tensorflow:global_step/sec: 2.63841
INFO:tensorflow:loss = 0.044804174, step = 97629 (37.902 sec)
INFO:tensorflow:lr = 0.0003349087 (37.902 sec)
INFO:tensorflow:global_step/sec: 2.64287
INFO:tensorflow:loss = 6.069026, step = 97729 (37.837 sec)
INFO:tensorflow:lr = 0.00033398

INFO:tensorflow:loss = 4.531274, step = 100541 (42.372 sec)
INFO:tensorflow:lr = 0.00030896385 (42.372 sec)
INFO:tensorflow:global_step/sec: 2.10958
INFO:tensorflow:loss = 6.923431, step = 100641 (47.403 sec)
INFO:tensorflow:lr = 0.00030810948 (47.403 sec)
INFO:tensorflow:Saving checkpoints for 100646 into ./ConvE/model.ckpt.
INFO:tensorflow:Loss for final step: 4.982877e-06.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ConvE/model.ckpt-100646
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[test] Raw Mean Rank: 556.9348
[test] Raw Hits@1: 28.82
[test] Raw Hits@3: 43.44
[test] Raw Hits@5: 50.36000000000001
[test] Raw Hits@10: 59.9
[test] Filtered Mean Rank: 543.2822
[test] Filtered Hits@1: 36.68
[test] Filtered Hits@3: 52.64
[test] Filtered Hits@5: 59.260000000000005
[test] Filtered Hits@10: 67.67999999999999
INFO:tensorflow:Calling model_fn.
I

INFO:tensorflow:lr = 0.00028975582
INFO:tensorflow:global_step/sec: 2.54103
INFO:tensorflow:loss = 0.361979, step = 102959 (39.355 sec)
INFO:tensorflow:lr = 0.00028895456 (39.356 sec)
INFO:tensorflow:global_step/sec: 2.53025
INFO:tensorflow:loss = 1.3306651, step = 103059 (39.521 sec)
INFO:tensorflow:lr = 0.0002881555 (39.522 sec)
INFO:tensorflow:global_step/sec: 2.51854
INFO:tensorflow:loss = 1.1161592, step = 103159 (39.706 sec)
INFO:tensorflow:lr = 0.00028735877 (39.706 sec)
INFO:tensorflow:global_step/sec: 2.53736
INFO:tensorflow:loss = 2.1049736, step = 103259 (39.411 sec)
INFO:tensorflow:lr = 0.00028656414 (39.411 sec)
INFO:tensorflow:global_step/sec: 2.36281
INFO:tensorflow:loss = 6.123993, step = 103359 (42.323 sec)
INFO:tensorflow:lr = 0.0002857717 (42.323 sec)
INFO:tensorflow:global_step/sec: 2.19965
INFO:tensorflow:loss = 1.7000254, step = 103459 (45.461 sec)
INFO:tensorflow:lr = 0.00028498156 (45.461 sec)
INFO:tensorflow:global_step/sec: 2.23767
INFO:tensorflow:loss = 0.543

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ConvE/model.ckpt-106176
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[test] Raw Mean Rank: 548.2126
[test] Raw Hits@1: 29.220000000000002
[test] Raw Hits@3: 44.18
[test] Raw Hits@5: 51.76
[test] Raw Hits@10: 61.06
[test] Filtered Mean Rank: 530.7274
[test] Filtered Hits@1: 38.0
[test] Filtered Hits@3: 54.50000000000001
[test] Filtered Hits@5: 61.28
[test] Filtered Hits@10: 68.74
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(10368, 200) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(200,) dt

INFO:tensorflow:global_step/sec: 2.47977
INFO:tensorflow:loss = 2.161105, step = 108689 (40.327 sec)
INFO:tensorflow:lr = 0.00024655985 (40.327 sec)
INFO:tensorflow:global_step/sec: 2.4913
INFO:tensorflow:loss = 0.67327327, step = 108789 (40.140 sec)
INFO:tensorflow:lr = 0.00024587807 (40.140 sec)
INFO:tensorflow:global_step/sec: 2.45683
INFO:tensorflow:loss = 1.3424199, step = 108889 (40.703 sec)
INFO:tensorflow:lr = 0.0002451982 (40.703 sec)
INFO:tensorflow:global_step/sec: 2.5602
INFO:tensorflow:loss = 1.0017723, step = 108989 (39.060 sec)
INFO:tensorflow:lr = 0.00024452014 (39.059 sec)
INFO:tensorflow:global_step/sec: 2.5563
INFO:tensorflow:loss = 0.386095, step = 109089 (39.119 sec)
INFO:tensorflow:lr = 0.00024384404 (39.119 sec)
INFO:tensorflow:global_step/sec: 2.54225
INFO:tensorflow:loss = 3.8194277, step = 109189 (39.335 sec)
INFO:tensorflow:lr = 0.00024316975 (39.335 sec)
INFO:tensorflow:global_step/sec: 2.5619
INFO:tensorflow:loss = 0.39097914, step = 109289 (39.034 sec)
INF